In [1]:

import numpy as np
import pandas as pd
import bokeh.plotting as bp
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from bokeh.models import  WheelZoomTool, ResetTool, PanTool


W = 590
H = 350
bp.output_notebook()

%matplotlib inline

Loading BokehJS ...

# Question 1: Gradient Descent for Multiple Linear Regression

## 1.a) Gradient of given equation

>\begin{align}
\nabla \ f({\beta})& =  (1/2) \ \nabla \|Y -X\beta \| _2^2   \\ 
& = \nabla  \left(\beta^T X^T X \beta + Y^T Y - 2 \beta^T X^T Y \right) \\ 
& = (X^T X \beta - X^T Y) \\
\end{align}

## 1.b) Step size
     
We want to choose a step length that matches the maximum underlying curvature of the given cost function. Since the function is twice differentiable and its gradient is Lipschitz continuous, the ideal value of the Lipschitz value will bound the eigenvalues of the Hessian ${X^TX}$. We will choose a step length that is equal to the largest eigenvalue of ${X^TX}$ (calculated in code below)


## 1.c) Stopping Rule
      
We will be stopping our algorithm when the improvement between two consecutive iterations is less than $0.000001$.       
      


## 1.d) MLR

In [2]:
mlr = pd.read_csv("MLR.csv", header = None)


In [3]:
#separating x & y from input data
x = mlr.iloc[:, :30].values
y = mlr.iloc[:, 30:].values.flatten()

In [4]:
def gradient_descent(x, y, iters, step):
    costs = []
    
    m = y.size # number of data points
     
    beta = np.zeros(x.shape[1] )  # random start
    
    beta_history = [beta] # to store all betas
    
    for i in range(iters):
        pred = np.dot(x, beta)
        error = pred - y 
        cost = np.sum(error ** 2) / (2 * m)
        costs.append(cost)
        gradient = x.T.dot(error)/m 
        beta = beta - step * gradient  # update
        beta_history.append(beta)
        if i>2:
            if abs(costs[i] - costs[i-1])<0.000001:
                break
        
    return beta_history, costs #, preds

Calculating Lipschitz constant for ideal step size

In [5]:
L = max(np.linalg.eig(x.T.dot(x))[0])

Assigning parameters for gradient descent calculation:

In [6]:
step = 1/L  # set step-size
iters = 5000 # set number of iterations
history, cost  = gradient_descent(x, y, iters, step)
beta = history[-1]

In [7]:
TOOLS = [WheelZoomTool(), ResetTool(), PanTool()]
x1, y1 = (zip(*enumerate(cost)))

fig = bp.figure(width=590, plot_height=350, 
                title='Loss over iterations (GD)',
                x_axis_label='Iteration',
                y_axis_label='Loss',
                tools=TOOLS )
s1 = fig.line(x1,y1, line_width=4, color='navy', alpha=0.5)
# remaining cost is tied directly to the noise of the distribution,
# ergo it's also a measure of fit
fig.title.text_font_size = '16pt'
fig.yaxis.axis_label_text_font_size = "14pt"
fig.xaxis.axis_label_text_font_size = "14pt"

bp.show(fig)


## 1.e) MSE 

In [8]:
true_beta = pd.read_csv("True_Beta.csv", header = None).values.flatten()

In [9]:
MSE = np.sum((true_beta - history[-1])**2)/30
print("Mean Squared Error: {:.4f}".format(MSE))

Mean Squared Error: 0.0011


# Question 2

## 2.a) Stochastic Gradient Descent

Defining single unit SGD function:
    
    

In [10]:
def stochastic_gradient_descent(x, y, iters, step):
    costs = []
    n = x.shape[1] # number of columns
    m = y.size # number of data points
    #x = np.c_[np.ones(x.shape[0]), x] 
    beta = np.zeros(n) 
    beta_history = [beta] # to store all thetas
    #preds = []
    for i in range(iters):
        cost = []
        for idx in range(m):
             
            pred = x[idx].dot(beta) 
            error = pred - y[idx]
            cost_idx = (error ** 2) /2
            cost.append(cost_idx)
            gradient = x[idx].T.dot(error)
            beta = beta - step * gradient  # update
            beta_history.append(beta)
        costs.append(np.mean(cost))
        
        if i>2:
            if abs(costs[i] - costs[i-1])<0.000001:
                break 

        
    return beta_history , costs  

Setting desired parameters :

In [11]:
step2 = 1/(L*x.shape[1]) # set step-size
iters2 = 5000 # set number of iterations
beta2_history, cost2  = stochastic_gradient_descent(x, y, iters2, step2)
beta2 = beta2_history[-1]


In [12]:
TOOLS = [WheelZoomTool(), ResetTool(), PanTool()]
x1, y1 = (zip(*enumerate(cost2)))
x1 = [x*1000 for x in x1]
fig = bp.figure(width=590, plot_height=350, 
                title='Cost over iterations (SGD)',
                x_axis_label='Iteration',
                y_axis_label='Cost',
                tools=TOOLS )
s1 = fig.line(x1,y1, line_width=4, color='navy', alpha=0.5)
# remaining cost is tied directly to the noise of the distribution,
# ergo it's also a measure of fit
fig.title.text_font_size = '16pt'
fig.yaxis.axis_label_text_font_size = "14pt"
fig.xaxis.axis_label_text_font_size = "14pt"

bp.show(fig)


## 2.b) Mini Batch Gradient Descent

Defining Mini Batch Gradient Descent Function:

In [13]:
def minibatch_gradient_descent(x, y, iters, step, b):
    costs = [] #contains averaged cost values (every 1000 iterations)
    cost_all = [] #Contains granular cost values
    n = x.shape[1] # number of columns
    m = y.size # number of data points
    
    beta = np.zeros(n) 
    beta_history = [beta] # to store all betas
    
    for i in range(iters):
        
        idx = np.random.randint(m, size=b)
        
        pred = x[idx].dot(beta) 
        error = pred - y[idx]
        cost = np.sum(error ** 2) / (2*m)
        cost_all.append(cost)
        gradient = x[idx].T.dot(error)/b
        beta = beta - step * gradient  # update
        beta_history.append(beta)
        if i%m==0:
            costs.append(np.mean(cost_all[-m:]))
            if len(costs)>1:
                if abs(costs[-1] - abs(costs[-2]))<0.000001:
                    break
    return beta_history, costs #costs

Batch Size: 10

In [14]:
b_10 = 10
step3_10 = b_10/(L*x.shape[1]) # set step-size
iters3_10 = 50000 # set number of iterations
history3_10, cost3_10 = minibatch_gradient_descent(x, y, iters3_10, step3_10, b_10)
beta3_10 = history3_10[-1]

TOOLS = [WheelZoomTool(), ResetTool(), PanTool()]
x1, y1 = (zip(*enumerate(cost3_10)))
x1 = [x*1000 for x in x1]

fig = bp.figure(width=590, plot_height=350, 
                title='Cost over iterations (batch: 10)',
                x_axis_label='Iteration',
                y_axis_label='Cost',
                tools=TOOLS )
s1 = fig.line(x1,y1, line_width=4, color='navy', alpha=0.5)
# remaining cost is tied directly to the noise of the distribution,
# ergo it's also a measure of fit
fig.title.text_font_size = '16pt'
fig.yaxis.axis_label_text_font_size = "14pt"
fig.xaxis.axis_label_text_font_size = "14pt"

bp.show(fig)



Batch Size: 25

In [15]:
b_25 = 25
step3_25 = b_25/(L*x.shape[1]) # set step-size
iters3_25 = 10000 # set number of iterations
history3_25, cost3_25 = minibatch_gradient_descent(x, y, iters3_25, step3_25, b_25)
beta3_25 = history3_25[-1]
TOOLS = [WheelZoomTool(), ResetTool(), PanTool()]
x1, y1 = (zip(*enumerate(cost3_25)))
x1 = [x*1000 for x in x1]
fig = bp.figure(width=590, plot_height=350, 
                title='Cost over iterations (batch: 25)',
                x_axis_label='Iteration',
                y_axis_label='Cost',
                tools=TOOLS )
s1 = fig.line(x1,y1, line_width=4, color='navy', alpha=0.5)
# remaining cost is tied directly to the noise of the distribution,
# ergo it's also a measure of fit
fig.title.text_font_size = '16pt'
fig.yaxis.axis_label_text_font_size = "14pt"
fig.xaxis.axis_label_text_font_size = "14pt"

bp.show(fig)



Batch Size: 100

In [16]:
b_100 = 100
step3_100 = b_100/(L*x.shape[1]) # set step-size
iters3_100 = 5000 # set number of iterations
history3_100, cost3_100 = minibatch_gradient_descent(x, y, iters3_100, step3_100, b_100)
beta3_100 = history3_100[-1]

TOOLS = [WheelZoomTool(), ResetTool(), PanTool()]
x1, y1 = (zip(*enumerate(cost3_100)))
x1 = [x*1000 for x in x1]
fig = bp.figure(width=590, plot_height=350, 
                title='Cost over iterations (batch: 100)',
                x_axis_label='Iteration',
                y_axis_label='Cost',
                tools=TOOLS )
s1 = fig.line(x1,y1, line_width=4, color='navy', alpha=0.5)
# remaining cost is tied directly to the noise of the distribution,
# ergo it's also a measure of fit
fig.title.text_font_size = '16pt'
fig.yaxis.axis_label_text_font_size = "14pt"
fig.xaxis.axis_label_text_font_size = "14pt"

bp.show(fig)



## 2.c) Mean Squared Errors

In [17]:
MSE2 = np.sum((true_beta - beta2)**2)/30
print("Mean Squared Error (SGD): {:.4f}".format(MSE2))

MSE3_10 = np.sum((true_beta - history3_10[-1])**2)/30
print("Mean Squared Error (batch 10 SGD): {:.4f}".format(MSE3_10))

MSE3_25 = np.sum((true_beta - history3_25[-1])**2)/30
print("Mean Squared Error (batch 25 SGD): {:.4f}".format(MSE3_25))

MSE3_100 = np.sum((true_beta - history3_100[-1])**2)/30
print("Mean Squared Error (batch 100 SGD): {:.4f}".format(MSE3_100))


Mean Squared Error (SGD): 0.0013
Mean Squared Error (batch 10 SGD): 0.0013
Mean Squared Error (batch 25 SGD): 0.0013
Mean Squared Error (batch 100 SGD): 0.0014


# Question 3

In [18]:
opca = pd.read_csv("OPCA.csv", header = None).values
true_eig = pd.read_csv("True_eigvector.csv", header = None).values

In [19]:
def oja_algo(mtx, eigs,  step = None, size = 20):
    
    mtxs = [mtx[i:i + size] for i in range(0, len(mtx), size)]
    weights = np.ones(size)*(1/np.sqrt(size))
    n = mtx.shape[0]/mtx.shape[1] #Number of matrices
    dist = []
    
    for idx in range(int(n)):
        mtx_sub = mtxs[idx] #Considering matrix A_i
        if step == None:
            step = 1/(100+idx)
            weights = weights + step*(np.dot(mtx_sub, weights))
            weights = weights/np.linalg.norm(weights)
            dist.append(1- (weights.dot(eigs)**2)[0])
        else:    
            weights = weights + step*(np.dot(mtx_sub, weights))
            weights = weights/np.linalg.norm(weights)
            dist.append(1- (weights.dot(eigs)**2)[0])
        

    return dist

## 3.a) Fixed Step Size (0.01)


In [20]:
distances_a = oja_algo(opca, true_eig,  step = 0.01) #Fixed Step Size

In [21]:
TOOLS = [WheelZoomTool(), ResetTool(), PanTool()]
x1, y1 = (zip(*enumerate(distances_a)))

fig = bp.figure(width=590, plot_height=350, 
                title='Distance over iterations (Fixed Step)',
                x_axis_label='Iteration',
                y_axis_label='Distance',
                tools=TOOLS )
s1 = fig.line(x1,y1, line_width=4, color='navy', alpha=0.5)
# remaining cost is tied directly to the noise of the distribution,
# ergo it's also a measure of fit
fig.title.text_font_size = '16pt'
fig.yaxis.axis_label_text_font_size = "14pt"
fig.xaxis.axis_label_text_font_size = "14pt"

bp.show(fig)



## 3.b) Variable Step Size

In [22]:

distances_b = oja_algo(opca, true_eig) #Variable Step Size

In [23]:
TOOLS = [WheelZoomTool(), ResetTool(), PanTool()]
x1, y1 = (zip(*enumerate(distances_b)))

fig = bp.figure(width=590, plot_height=350, 
                title='Distance over iterations (Variable Step)',
                x_axis_label='Iteration',
                y_axis_label='Distance',
                tools=TOOLS )
s1 = fig.line(x1,y1, line_width=4, color='navy', alpha=0.5)
# remaining cost is tied directly to the noise of the distribution,
# ergo it's also a measure of fit
fig.title.text_font_size = '16pt'
fig.yaxis.axis_label_text_font_size = "14pt"
fig.xaxis.axis_label_text_font_size = "14pt"

bp.show(fig)

